# Table of Contents

## 1. Introduction

The purpose of this notebook is to provide a starting place for running HOOMD-blue molecular dynamics simulations. It is recommended that you read the official HOOMD-blue tutorial (<https://hoomd-blue.readthedocs.io/en/v3.0.1/tutorial/01-Introducing-Molecular-Dynamics/01-Molecular-Dynamics-Simulations.html>) that contains some basic information that can help you to understand the basic set-up of the simulation scripts before going through this notebook. This document will hopefully help you to remember and get more familiar with the HOOMD-blue classes/functions and Python syntax to use them.


## 2. Ensuring that you have the necessary packages to go through this tutorial <a name="system_check"></a>

Going through this notebook will require you to have certain Python packages installed on your machine. Assuming you have them set-up within your conda environment, activate the environment and check for the packages with `conda list | grep "hoomd\|freud*\|gsd\|matplotlib"`.

The output should resemble the following:

|  |  |  |  |
| --- | --- | --- | --- |
| freud | 2.8.0 | py310he7ab2d0_0 | conda-forge |
| gsd | 2.5.2 | py310h7f5fb2b_0 | conda-forge |
| hoomd | 3.0.0 | cpu_py310h89b14bd_1 | conda-forge |
| matplotlib | 3.5.1 | py310hecd8cb5_1 |  |
| matplotlib-base | 3.5.1 | py310hfb0c5b7_1 |  |
| matplotlib-inline | 0.1.2 | pyhd3eb1b0_2 |  |

Unless there is a major version difference ex. V2.x.x vs V3.x.x, there shouldn't be a difference in the syntax, so don't worry about minor details of the output. If there has been a major version change for any of the packages, please raise an issue on the github repository for it to be updated.

If all four packages are installed, start up an interactive Python session with `python` and type the following for the basic functions of the simulation to ensure that your packages are functioning correctly without conflicts.

```python
import hoomd, freud, gsd.hoomd, matplotlib.pyplot as plt
hoomd.device.CPU()
freud.Box.cube(10)
gsd.hoomd.Snapshot()
plt.figure()
```

If all works without any error, proceed to the next section. If not, you may need to reinstall the malfunctioning packages.


## 3. Initializing a system - FCC lattice

There are a few ways to initialize a molecular dynamics system. For example, to create an FCC lattice, you can choose from:

1. custom python script to provide molecule positions
2. freud unit cell functions
3. other specialized packages as suggested by HOOMD-blue developers
    * mbuild (<https://mbuild.mosdef.org/en/stable/>)
    * Foyer (<https://foyer.mosdef.org/en/stable/>)

This notebook will focus on using the first two options.


### 3-1 Building an FCC unit cell using custom python scripts

The process is broken down to the following:

1. Define the system properties
    1. particle size
    2. density
    3. dimension of the simulation box (x:y:z)
2. Define a unit cell from the given parameters
3. clone the unit cell to produce coordinate points
4. output a gsd snapshot to be called into HOOMD-blue

See below for a code block that generates an FCC lattice snapshot with 20000 particles with diameter of 1.0 and Lennard-Jones potential with epsilon of 6 kT, system density of 0.60, and box dimensions of (x:y:z=1:1:1).


In [10]:
import numpy as np
import hoomd
import gsd.hoomd

############### system parameters ###############
N = int(20000) # number of particles
sigma = float(1.0) # particle diameter
phi = float(0.60) # system density
boxDim = np.array([1,1,1,0,0,0], dtype=int) # simulation box dimensions

############### derived parameters ###############
partVol = float(np.pi * (sigma) ** 3 / 6) # volume of a single particle
totPartVol = float(partVol * N) # volume of all particles
boxVol = float(totPartVol / phi) # volume of the simulation box with the given system density
targetBox = boxDim * (boxVol / np.prod(boxDim[0:3])) ** (1 / 3) # box configuration

############### initiate a gsd snapshot ###############
snapshot = gsd.hoomd.Snapshot() # define a snapshot
snapshot.particles.types=['A'] # add a single species of particles

############### calculate number of unit cell repeat ###############
cellSideLen = sigma * ((2 * np.pi) / (3 * phi)) ** (1/3) # unit cell side length
repeatTestVal = np.floor((boxDim/cellSideLen)).astype('int64') # number of cells on each side
uCellRepeat = repeatTestVal if 4 * np.prod(repeatTestVal[0:3]) >= N \
              else np.ceil((boxDim/cellSideLen)).astype('int64') # test if the number of cells was an underestimate, and if so, overestimate
temp = ( # template unit cell with a corner at the origin
    np.array([0, 0, 0]),
    np.array([cellSideLen/2, cellSideLen/2, 0]),
    np.array([cellSideLen/2, 0, cellSideLen/2]),
    np.array([0, cellSideLen/2, cellSideLen/2])
)

cellLengths = cellSideLen * (uCellRepeat+[1, 1, 1, 0, 0, 0]) #overestimate the box dimensions to ensure all particles are within the box
snapshot.configuration.box = cellLengths

############### iterate over the number of repeat and populate position array ###############
pos = []
for i in range(uCellRepeat[0]): # loop over the first dimension
    for j in range(uCellRepeat[1]): # loop over the second dimension
        for k in range(uCellRepeat[2]): # loop over the third dimension
            currentLoc = np.array([ # translation vector
                -uCellRepeat[0] * cellSideLen/2 + i * cellSideLen + (snapshot.configuration.box[0] - cellLengths[0])/2,
                -uCellRepeat[1] * cellSideLen/2 + j * cellSideLen + (snapshot.configuration.box[1] - cellLengths[0])/2,
                -uCellRepeat[2] * cellSideLen/2 + k * cellSideLen + (snapshot.configuration.box[1] - cellLengths[0])/2
            ])
            for l in temp: # append each position value after translation
                if len(pos) < N:
                    pos.append(l+currentLoc)

snapshot.particles.position = pos # set particle positions



<class 'numpy.ndarray'>


### 3-2 Building an FCC unit cell using freud unit cell functions

Using frued requires less creativity as it automates the process. Refer

The advantages of using a custom script are that

1. you know exactly what is going on with the initialization process and thus can predict/correct the initialization behaviors
2. you can define more than the basic lattices provided by the software packages

and the disadvantages are that

1. you need to make your own script
2. upon syntax change, you need to maintain and update the code yourself